In [1]:
import pandas as pd
import sqlite3
import csv

In [2]:
# Specify the path to your CSV file
csv_file_path = 'covid_19_clean_complete.csv'

# Open the CSV file
with open(csv_file_path, 'r') as file:
    # Create a CSV reader object
    csv_reader = csv.reader(file)

#     # Iterate over each row in the CSV file
#     for row in csv_reader:
#         # Each row is a list where each element corresponds to a column in the CSV
#         print(row)

In [4]:
# Load covid report data into a DataFrame
df = pd.read_csv("covid_19_clean_complete.csv")

# Filter and clean the DataFrame
df_transform = df[(df['Confirmed'] >1) &
                 (df['Deaths'] >= 0)]
        

df_transform = df_transform.copy()
df_transform.rename(columns={'Province/State': 'State', 'Country/Region': 'Country','WHO Region':'WHORegion'}, inplace=True)
#df_transform = df.dropna()
df_transform


,State,Country,Lat,Long,Date,Confirmed,Deaths,Recovered,Active,WHORegion
49,Beijing,China,40.182400,116.414200,2020-01-22,14,0,0,14,Western Pacific
50,Chongqing,China,30.057200,107.874000,2020-01-22,6,0,0,6,Western Pacific
53,Guangdong,China,23.341700,113.424400,2020-01-22,26,0,0,26,Western Pacific
54,Guangxi,China,23.829800,108.788100,2020-01-22,2,0,0,2,Western Pacific
56,Hainan,China,19.195900,109.745300,2020-01-22,4,0,0,4,Western Pacific
...,...,...,...,...,...,...,...,...,...,...
49063,NaN,Sao Tome and Principe,0.186400,6.613100,2020-07-27,865,14,734,117,Africa
49064,NaN,Yemen,15.552727,48.516388,2020-07-27,1691,483,833,375,Eastern Mediterranean
49065,NaN,Comoros,-11.645500,43.333300,2020-07-27,354,7,328,19,Africa
49066,NaN,Tajikistan,38.861000,71.276100,2020-07-27,7235,60,6028,1147,Europe


In [5]:
#Create/connect to a SQLite database
conn = sqlite3.connect('covid.db')
cursor = conn.cursor()


In [6]:
create_sql = '''
    DROP TABLE IF EXISTS country 
'''
cursor.execute(create_sql)

In [7]:
create_sql = '''
    CREATE TABLE IF NOT EXISTS country (
        "RowID" INTEGER PRIMARY KEY AUTOINCREMENT,
        "Country" TEXT,
        "State" TEXT,
        "Lat" REAL,
        "Long" REAL,
        "Date" TEXT,
        "Confirmed" INTEGER, 
        "Deaths" INTEGER,
        "Recovered" INTEGER,
        "Active" INTEGER,
        "WHORegion" TEXT
      
    )
'''
cursor.execute(create_sql)

In [8]:
# Insert data into the "country" table
#df_transform[['Country','State', 'Lat', 'Long','Date', 'Confirmed', 'Deaths', 'Recovered', 'Active', 'WHO Region']].to_sql('country', conn, index=False, if_exists='replace')

In [9]:
# # Insert data into the "country" table
df_transform[['Country','State','Lat', 'Long','Date', 'Confirmed', 'Deaths', 'Recovered', 'Active', 'WHORegion']].to_sql('country', conn, index=False,if_exists='append')

37800

In [10]:
# Replace 'your_table' with the actual name of the table you want to check
table_name = 'country'

# Fetch data from the table
query = f"SELECT * FROM {table_name}"
existing_data = pd.read_sql_query(query, conn)

# Print or examine the existing data
print(existing_data)

       RowID                Country      State        Lat        Long  \
0          1                  China    Beijing  40.182400  116.414200   
1          2                  China  Chongqing  30.057200  107.874000   
2          3                  China  Guangdong  23.341700  113.424400   
3          4                  China    Guangxi  23.829800  108.788100   
4          5                  China     Hainan  19.195900  109.745300   
...      ...                    ...        ...        ...         ...   
37795  37796  Sao Tome and Principe       None   0.186400    6.613100   
37796  37797                  Yemen       None  15.552727   48.516388   
37797  37798                Comoros       None -11.645500   43.333300   
37798  37799             Tajikistan       None  38.861000   71.276100   
37799  37800                Lesotho       None -29.610000   28.233600   

             Date  Confirmed  Deaths  Recovered  Active              WHORegion  
0      2020-01-22         14       0      

In [11]:
#close connection
conn.commit()
conn.close()